<a href="https://colab.research.google.com/github/abubakar-zohaib1415/human12/blob/main/Browsing_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q -U langgraph langchain_core langchain langchain_google_genai tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.8 MB/s eta 0:00:00


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import START, END , StateGraph
from google.colab import userdata
from typing_extensions import TypedDict
from langchain_core.messages.ai import AIMessage
from langgraph.graph.state import CompiledStateGraph
from IPython.display import Image, display

In [3]:
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
TAVILY_API_KEY =  userdata.get('TAVILY_API_KEY')

llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-flash",
    api_key = GEMINI_API_KEY
)

In [4]:
class Browser(TypedDict):
  prompt : str
  browsing : str
  answer : str

In [5]:
def chatbot(state: Browser):
    print("---Web_Browser---", state)
    prompt = state["prompt"]
    ai_msg: AIMessage = llm.invoke(prompt)
    return {"🤖 ChatBot": ai_msg.content}

greet_message = chatbot(Browser(prompt="Hello I am Abu Bakar"))
print(greet_message)

---Web_Browser--- {'prompt': 'Hello I am Abu Bakar'}
{'🤖 ChatBot': "Hello Abu Bakar, it's nice to meet you.  How can I help you today?\n"}


In [17]:
from tavily import TavilyClient

# Step 1. Instantiating your TavilyClient
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# Step 2. Executing a simple search query
response = tavily_client.search("Who is Leo Messi?")

# Step 3. That's it! You've done a Tavily Search!
print(response)


{'query': 'Who is Leo Messi?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Lionel Messi Facts | Britannica', 'url': 'https://www.britannica.com/facts/Lionel-Messi', 'content': 'Lionel Messi, an Argentine footballer, is widely regarded as one of the greatest football players of his generation. Born in 1987, Messi spent the majority of his career playing for Barcelona, where he won numerous domestic league titles and UEFA Champions League titles. Messi is known for his exceptional dribbling skills, vision, and goal', 'score': 0.8351749, 'raw_content': None}, {'title': 'Lionel Messi | Biography, Competitions, Wins and Medals', 'url': 'https://olympics.com/en/athletes/lionel-messi', 'content': '“The Olympic gold in 2008 is the win that I value the most because it is a tournament that you may play only once in your life and involves many athletes from different disciplines” - Lionel Messi, Spanish Esquire\nFeatured\nLatest News\nOlympic Results\nAthlete

In [19]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, trim_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata


# Define trimmer
trimmer = trim_messages(strategy="last", max_tokens=5, token_counter=len)

# Define the workflow
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    system_prompt = (
    f"You Are A Helpfull Assistant. For Searching On Website You Not Need To Search For Search Engine You Need To Answer Poliely Your Main Is For You Will Answer for searching car price and car details we will use tavily will give you data and you will represent it to user for any searching hee take this {user_input}"
    )
    messages = [SystemMessage(content=system_prompt)] + trimmed_messages
    response = llm.invoke(messages)
    return {"messages": response}

# Add node and edge to workflow
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add in-memory checkpointing
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Chat interaction loop
print("Welcome to Clothes Customer Support. Type 'exit' to end the chat.")

messages = []
while True:
    # Get user input
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Assistant: Goodbye!")
        break

    # Add user's message to history
    messages.append(HumanMessage(content=user_input))

    # Invoke the app
    response = app.invoke({"messages": messages}, config={"configurable": {"thread_id": "chat-session"}})

    # Get AI response
    ai_response = response["messages"][-1].content
    print(f"Assistant: {ai_response}")

    # Add AI's response to the history
    messages.append(AIMessage(content=ai_response))


Welcome to Clothes Customer Support. Type 'exit' to end the chat.
You: Hello
Assistant: Hello!  How can I help you today?  I can assist you with finding car prices and details using the Tavily data.  Please let me know what you're looking for.

You: Yes I want Audi V8
Assistant: Okay, I'd be happy to help you find information on the Audi V8.  However, to give you the most accurate and relevant information from Tavily, I need some more specifics.  The Audi V8 has been produced over many years and in various models.  Could you please tell me:

* **Which model year are you interested in?** (e.g., 2023, 2018, 2000)  This is crucial as prices and features will vary significantly.
* **Which body style?** (e.g., Sedan, Coupe, Convertible, Quattro)
* **Any specific trim level?** (e.g., S8, RS6, etc.)  Different trim levels will have different features and prices.
* **Are you interested in new or used cars?**  This will greatly affect the price range.
* **Location (if searching for used cars)?*

KeyboardInterrupt: Interrupted by user